In [1]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()

Settings从哪里加载？: config.settings.development
项目根目录对么？: /home/oscarwang/BidPilot_new/backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- apps.testground
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


## PART-1. 进入Outline_L1 分析
<font color="orange">
事实证明，对于长文本而言，让大模型一次性提取多个层级的标题时，表现并不好，质量也不稳定。 <br>
因此，我们决定分步骤提取，先提取一级标题，再提取二级标题，最后提取三级标题。 <br>
为了接下去的测试，我们需要：<br> 
1. 项目容器： 测试项目1 <br>
2. tender_file_extraction 已经ready <br>
3. 模型输入： context, instruction, supplement, output_format, prompt_template, index_path_map, 这里都是即时生成，不做存储。<br>
4. 调用模型的函数：从task中截取和简化。 <br>
4. 标题应用函数： <br>
5. 章节的截取函数： TiptapUtils.extract_indexed_paragraphs， 在prepare_context中已调用<br>
6. 任务结果L1： 空 <br>  有了 
7. 任务结果L2： 空 <br>  有了
8. 任务结果L3： 空 <br>  有了
</font>

#### <font color="red"> 1. 准备好测试的项目容器，招标文件内容，用于存储任务结果的字段</font>

In [2]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='测试项目1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
# --- 如果project.tender_file_extraction 为空，则需要从关联的docx_extraction_task中获取
# extraction_task = Task.objects.get(stage__project=project, type=TaskType.DOCX_EXTRACTION_TASK)
# print(extraction_task.docx_tiptap)
# print(type(extraction_task.docx_tiptap))
# pprint(extraction_task.docx_tiptap)
# project = Project.objects.get(project_name='测试项目1')
# project.tender_file_extraction = extraction_task.docx_tiptap
# project.save()
# project = Project.objects.get(project_name='测试项目1')
pprint(project.tender_file_extraction)

{'content': [{'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '北京京铁运恒采购供应站有限公司',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '2024',
                           'type': 'text'},
                          {'text': ' ', 'type': 'text'},
                          {'marks': [{'type': 'bold'}],
                           'text': '年端午节物资采购项目',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                           'text': '招标文件',
                           'type': 'text'}],
              'type': 'paragraph'},
             {'attrs': {'text_align': 'left'},
              'content': [{'marks': [{'type': 'bold'}],
                          

In [3]:
# 存储位就绪
print(f"outline_L1存储：{project.outline_L1}")
print(f"outline_L2存储：{project.outline_L2}")
print(f"outline_L3存储：{project.outline_L3}")
print(f"outline_L1存储：{project.index_path_map_L1}")
print(f"outline_L2存储：{project.index_path_map_L2}")
print(f"outline_L3存储：{project.index_path_map_L3}")

outline_L1存储：[
    {"index": 45, "level": 1, "title": "第一章 招标公告"},
    {"index": 93, "level": 1, "title": "第二章 招标需求"},
    {"index": 119, "level": 1, "title": "第三章 投标人须知"},
    {"index": 298, "level": 1, "title": "第四章 评标办法及评分标准"},
    {"index": 350, "level": 1, "title": "第五章 合同条款及格式"},
    {"index": 484, "level": 1, "title": "第六章 投标文件格式"}
]
outline_L2存储：None
outline_L3存储：None
outline_L1存储：{'0': [0], '1': [1], '2': [2], '3': [3], '4': [4], '5': [5], '6': [6], '7': [7], '8': [8], '9': [9], '10': [10], '11': [11], '12': [12], '13': [13], '14': [14], '15': [15], '16': [16], '17': [17], '18': [18], '19': [19], '20': [20], '21': [21], '22': [22], '23': [23], '24': [24], '25': [25], '26': [26], '27': [27], '28': [28], '29': [29], '30': [30], '31': [31], '32': [32], '33': [33], '34': [34], '35': [35], '36': [36], '37': [37], '38': [38], '39': [39], '40': [40], '41': [41], '42': [42], '43': [43], '44': [44], '45': [45], '46': [46], '47': [47], '48': [48], '49': [49], '50': [50], '51': [51], '52

#### <font color="red"> 2. 准备大模型的输入，并调用模型</font>

In [4]:
# 借用outline_analysis_L1.py 的函数， 进行任务准备 
# from apps.projects.services.tasks_preparation.outline_analysis_L1 import OutlineAnalysisL1
# prompt_template_L1 = OutlineAnalysisL1(project.id)._build_prompt_template()
# context_L1, index_path_map_L1 = OutlineAnalysisL1(project.id)._prepare_context(project)
# instruction_L1 = OutlineAnalysisL1(project.id)._prepare_instruction()
# supplement_L1 = OutlineAnalysisL1(project.id)._prepare_supplement()
# output_format_L1 = OutlineAnalysisL1(project.id)._prepare_output_format()
# LLM_config_L1 = OutlineAnalysisL1(project.id)._build_llm_config()

# project.index_path_map_L1 = index_path_map_L1
# project.save()

# 集成时，用以下代替，模型传参 会先存储到数据库。
from apps.projects.services.tasks_preparation.outline_analysis_L1 import OutlineAnalysisL1
params_L1=OutlineAnalysisL1(project.id).output_params()


In [5]:
from pprint import pprint
pprint(params_L1['prompt_template'])
pprint(params_L1['context'])
pprint(params_L1['instruction'])
pprint(params_L1['supplement'])
pprint(params_L1['output_format'])
pprint(params_L1['llm_config'])
pprint(params_L1['index_path_map'])



('\n'
 '你将执行以下任务：\n'
 '\n'
 '【任务目标】\n'
 '{instruction}\n'
 '\n'
 '【输出格式】\n'
 '{output_format}\n'
 '\n'
 '以下是你将使用的内容：\n'
 '\n'
 '【材料A：主要上下文】\n'
 '{context}\n'
 '\n'
 '【材料B：补充信息】\n'
 '{supplement}\n'
 '\n'
 '请严格根据材料A和B完成任务。\n'
 '\n')
('content: 北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目 | index: 0\n'
 'content: 招标文件 | index: 1\n'
 'content: 项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油） | index: 2\n'
 'content: 项目编号：DLXM-2024-148-01 | index: 3\n'
 'content: 招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司 | index: 4\n'
 'content: 2024 年 3 月 | index: 5\n'
 'content: 目 录 | index: 6\n'
 'content: 第一章 招标公告 2 | index: 7\n'
 'content: 第二章 招标需求 5 | index: 8\n'
 'content: 第三章 投标人须知 8 | index: 9\n'
 'content: 前附表 8 | index: 10\n'
 'content: 一、总则 12 | index: 11\n'
 'content: 二、招标文件 13 | index: 12\n'
 'content: 三、投标文件的编制 14 | index: 13\n'
 'content: 四、开标 18 | index: 14\n'
 'content: 五、评标 19 | index: 15\n'
 'content: 六、定标 21 | index: 16\n'
 'content: 七、合同授予 21 | index: 17\n'
 'content: 八、履约保证金 21 | index: 18\n'
 '

In [6]:
# # 模拟的prompt, 
simulated_prompt, formatted_message = OutlineAnalysisL1(project.id).simulate_prompt()
pprint(formatted_message)

[{'content': '你是一个专业的招标文档分析助手，帮助用户分析文档的结构和内容。', 'role': 'system'},
 {'content': '\n'
             '你将执行以下任务：\n'
             '\n'
             '【任务目标】\n'
             '\n'
             '我会提供某文档的完整文本（材料A），每条数据包含 content（文本）和 index（位置索引）。\n'
             '\n'
             '请完成以下任务：\n'
             '1. 识别文档最高层级的标题。\n'
             '2. 请仅识别正文中的标题，忽略目录、封面页的标题，忽略附件中的重复章节名称。\n'
             '\n'
             '\n'
             '\n'
             '【输出格式】\n'
             '\n'
             '        \n'
             '- 只输出符合JSON格式的数据，不要添加解释、注释或 Markdown 标记。\n'
             '- 示例：\n'
             '[\n'
             '    {"index": int, "level": int, "title": str}, \n'
             '    {"index": int, "level": int, "title": str}\n'
             ']\n'
             '- 一个标题一条数据， 只输出最高层级的标题。\n'
             '\n'
             '\n'
             '\n'
             '以下是你将使用的内容：\n'
             '\n'
             '【材料A：主要上下文】\n'
             'content: 北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目 | index: 0\n'
             'c

#### <font color="red"> 3. 大模型的调用分析 Outline_L1</font>
<font color="orange" size="3">
- 分析获得outline_L1 <br>
- 存储结果到project.outline_L1 <br>
</font>

In [9]:
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
analyzer = LLMService(
    llm_config = LLMConfig().from_model(params_L1['llm_config']),
    context = params_L1['context'], 
    instruction = params_L1['instruction'], 
    supplement = params_L1['supplement'], 
    output_format = params_L1['output_format'], 
    prompt_template = params_L1['prompt_template'])
outline_L1 = await analyzer.process()
pprint(outline_L1)

# 将outline_L1 存储到project.outline_L1
# print(type(outline_L1), outline_L1)
# project.outline_L1 = outline_L1
# project.save()


INFO 2025-04-17 14:20:58,655 LLMchain Final prompt:
messages=[SystemMessage(content='你是一个专业的招标文档分析助手，帮助用户分析文档的结构和内容。', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n你将执行以下任务：\n\n【任务目标】\n\n我会提供某文档的完整文本（材料A），每条数据包含 content（文本）和 index（位置索引）。\n\n请完成以下任务：\n1. 识别文档最高层级的标题。\n2. 请仅识别正文中的标题，忽略目录、封面页的标题，忽略附件中的重复章节名称。\n\n\n\n【输出格式】\n\n        \n- 只输出符合JSON格式的数据，不要添加解释、注释或 Markdown 标记。\n- 示例：\n[\n    {"index": int, "level": int, "title": str}, \n    {"index": int, "level": int, "title": str}\n]\n- 一个标题一条数据， 只输出最高层级的标题。\n\n\n\n以下是你将使用的内容：\n\n【材料A：主要上下文】\ncontent: 北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目 | index: 0\ncontent: 招标文件 | index: 1\ncontent: 项目名称：北京京铁运恒采购供应站有限公司 2024 年端午节 物资采购项目（第一包：一级压榨花生油） | index: 2\ncontent: 项目编号：DLXM-2024-148-01 | index: 3\ncontent: 招 标 人：北京京铁运恒采购供应站有限公司 代理机构：北京中外建工程管理有限公司 | index: 4\ncontent: 2024 年 3 月 | index: 5\ncontent: 目 录 | index: 6\ncontent: 第一章 招标公告 2 | index: 7\ncontent: 第二章 招标需求 5 | index: 8\ncontent: 第三章 投标人须知 8 | index: 9\ncontent: 前附表 8 | in

HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
[
    {"index": 45, "level": 1, "title": "第一章 招标公告"},
    {"index": 93, "level": 1, "title": "第二章 招标需求"},
    {"index": 119, "level": 1, "title": "第三章 投标人须知"},
    {"index": 298, "level": 1, "title": "第四章 评标办法及评分标准"},
    {"index": 350, "level": 1, "title": "第五章 合同条款及格式"},
    {"index": 484, "level": 1, "title": "第六章 投标文件格式"}
]

INFO 2025-04-17 14:21:08,814 LLMchain Output tokens: 172


('[\n'
 '    {"index": 45, "level": 1, "title": "第一章 招标公告"},\n'
 '    {"index": 93, "level": 1, "title": "第二章 招标需求"},\n'
 '    {"index": 119, "level": 1, "title": "第三章 投标人须知"},\n'
 '    {"index": 298, "level": 1, "title": "第四章 评标办法及评分标准"},\n'
 '    {"index": 350, "level": 1, "title": "第五章 合同条款及格式"},\n'
 '    {"index": 484, "level": 1, "title": "第六章 投标文件格式"}\n'
 ']')


#### <font color="red"> 4. 将Outline_L1 应用到tender_file_extraction中</font>
<font color="orange" size="3">
- 将Outline_L1 应用到tender_file_extraction <br>
- 我们需要解决：<br>
    1）outline_L1 从JSON str 转 JSON Dict？<br>
    2）如何将结果应用到tiptap JSON的 内容中？<br>
- 存储结果到project.tender_file_extraction <br>
</font>



In [ ]:
# 从数据库中获取数据
project = Project.objects.get(project_name='测试项目1')
outline_L1 = project.outline_L1
index_path_map_L1 = project.index_path_map_L1
print(f"outline_L1：{type(outline_L1)}, {outline_L1}")
print(f"index_path_map_L1：{type(index_path_map_L1)}, {index_path_map_L1}")


In [ ]:
# 将outline_L1 从JSON str 转 JSON Dict 
import json
outline_L1_dict = json.loads(outline_L1)
pprint(outline_L1_dict)
pprint(index_path_map_L1)

In [10]:
# 将outline_L1 应用到tender_file_extraction中
from apps.projects.tiptap.helpers import TiptapUtils
updated_tiptap_content = TiptapUtils.update_titles_from_list(project.tender_file_extraction, outline_L1_dict, index_path_map_L1)
project.tender_file_extraction_L1 = updated_tiptap_content
project.save()

In [ ]:
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(updated_tiptap_content)
pprint(headings)
print(TiptapUtils.print_headings(updated_tiptap_content))

## PART-2. 开始执行Outline_L2 分析
<font color="orange" size="3">
- 带有L1标题的tiptap JSON 内容已经ready <br>
- 我们需要解决：<br>
    1）提取章节？-> 在原文 基础上 提取段落内容（str）， 根据章节分块， 好处是,index被保留。<br>
    2）提供章节段落内容，重复类似Outline_L1的分析: <br>
        a. 准备容器，更新的文件内容，存储任务结果字段？<br>
        b. 准备大模型的输入，并调用模型？ (这个过程，需要考虑并发分析，提高效率)<br>
        c. 将结果应用到tiptap JSON的 内容中？<br>
        d. 存储结果到project.tender_file_extraction <br>
</font>

In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 从数据库中获取数据
from apps.projects.models import Project, Task, TaskType
project = Project.objects.get(project_name='测试项目1')
# pprint(project.tender_file_extraction_L1)

# 提取章节
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(project.tender_file_extraction_L1)
print(headings)

#### <font color="red"> 1. 通过outline_analysis_L2 进行传参准备</font>
<font color="orange" size="3">
- 在outline_analysis_L2 中， 根据章节分块， index被保留。<br>
- 使用了TiptapUtils.extract_chapters 工具，返回 字典对象chapter_set{}， 包含chapters 和 index_path_map 两个键名<br>
- 在outline_analysis_L2 中， 使用chapter_set['chapters'] for循环 取得chapter['paragraphs'] 作为context<br>
- 在outline_analysis_L2 的输出，将每个章节分析所需的传参 单独打包， 并返回 列表对象param_set[]<br>
</font>

In [3]:
# 通过集成的outline_anlaysis_L2 获取 大模型分析所需的 输入参数 ，并有 模拟的prompts
from apps.projects.services.tasks_preparation.outline_analysis_L2 import OutlineAnalysisL2
from pprint import pprint
params_L2_set, index_path_map_L2 = OutlineAnalysisL2(project.id).output_params()
simulated_prompt_set, formatted_prompt_set = OutlineAnalysisL2(project.id).simulate_prompt()

In [15]:
contexts = []
for params in params_L2_set:
    contexts.append(params['context'])

In [ ]:
print(len(contexts))
print(contexts[4])

In [ ]:
pprint(formatted_prompt_set[6])

#### <font color="red"> 2. 大模型逐个章节进行分析</font>

In [ ]:
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
from apps.projects.services.LLM_service.LLMcontainer import LLMService
results_L2 = []
for params in params_L2_set:
    analyzer = LLMService(
        llm_config = LLMConfig().from_model(params['llm_config']),
        context = params['context'], 
        instruction = params['instruction'], 
        supplement = params['supplement'], 
        output_format = params['output_format'], 
        prompt_template = params['prompt_template']
    )
    result = await analyzer.process()
    results_L2.append(result)

In [ ]:
from apps.projects.services.LLM_service._llm_data_types import LLMConfig
from apps.projects.services.LLM_service.LLMcontainer import LLMService
results2_L2 = []
for params in params_L2_set[5:6]:
    analyzer2 = LLMService(
        llm_config = LLMConfig().from_model(params['llm_config']),
        context = params['context'], 
        instruction = params['instruction'], 
        supplement = params['supplement'], 
        output_format = params['output_format'], 
        prompt_template = params['prompt_template']
    )
    result2 = await analyzer2.process()
    results2_L2.append(result2)

In [ ]:
pprint(results2_L2[0])